In [2]:
import json
import csv
from collections import defaultdict

# File paths
kitle_csv_path = r'C:\Users\User\Desktop\compitission\kitle_Predictions.csv'
kalsifikasyon_csv_path = r'C:\Users\User\Desktop\compitission\kalsifikasyon_Predictions.csv'
json_input_path = r'C:\Users\User\Desktop\Test10\outputs.json'
json_output_path = r'C:\Users\User\Desktop\compitission\outputs_updated.json'

# Load JSON file
with open(json_input_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Function to process CSV and update dictionary with "kitle" or "kalsifikasyon" data
def process_csv(csv_path, category_type, patient_predictions):
    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Extract patient ID and image type
            image_name = row['Image Name']
            patient_id, image_type = image_name.split('_')[0], image_name.split('_')[1].split('.')[0]
            image_type = image_type.lower()
            image_type = image_type + ".dcm"
            
            # Initialize patient if not already in the JSON
            if patient_id not in patient_predictions:
                patient_predictions[patient_id] = {}
                
            # Initialize image type if not already there
            if image_type not in patient_predictions[patient_id]:
                patient_predictions[patient_id][image_type] = {"kitle": [], "kalsifikasyon": []}

            # Category-specific processing
            if row['Category'] != 'normal':
                # Bounding box and confidence
                bounding_box = row['Bounding Box (x1,y1;x2,y2;x3,y3;x4,y4)']
                confidence = row['Confidence Score']
                # Add to the corresponding category in the image type
                patient_predictions[patient_id][image_type][category_type].append({
                    "kutu": bounding_box,
                    "guven": confidence
                })

# Dictionary to store patient predictions
patient_predictions = defaultdict(dict)

# Process CSV files
process_csv(kitle_csv_path, 'kitle', patient_predictions)
process_csv(kalsifikasyon_csv_path, 'kalsifikasyon', patient_predictions)

# Update the original JSON with the processed data
for patient_id, images in patient_predictions.items():
    if patient_id in data['tahminler']:
        for image_type, findings in images.items():
            # Ensure "dosya" exists in the JSON structure
            if 'dosya' not in data['tahminler'][patient_id]:
                data['tahminler'][patient_id]['dosya'] = {}
            # Update with "kitle" and "kalsifikasyon" if present
            data['tahminler'][patient_id]['dosya'][image_type] = findings

# Write updated JSON to a new file
with open(json_output_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"Updated JSON file saved to: {json_output_path}")


Updated JSON file saved to: C:\Users\User\Desktop\Test10\outputs_updated.json
